# IQ-Modulation und -Demodulation #

Ein großer Vorteil einer digitalen IQ-Mischstufe ist die hohe Präzision beim digitalen Mischen, die mit analogen IQ-Mischern kaum erreicht werden kann.


Wie man sich leicht vorstellen kann, sind solche Amplituden- und Phasengenauigkeiten mit analogen Schaltungen, die mehrere Amateurfunkbänder abdecken sollen und Temperaturschwankungen, Vibrationen und Bauteilealterung ausgesetzt sind, ziemlich unerreichbar.

## So funktioniert ein IQ-Empfangsmischer ##

An der Antenne kommt ein Signal mit der Frequenz $ \omega_{1} $ an.
Dieses Signal soll den zeitlichen Verlauf

$ U_{Ant}(t) = \cos ( \omega_{1} t ) $

haben und am Eingang des IQ-Mischers anliegen.
Es ist hierbei egal, ob das Signal bereits digitalisiert wurde oder ob es sich um ein Analogsignal handelt, die Mathematik ist die gleiche.
Die Amplitude ist auch erst mal egal; wir setzen die Amplitude auf 1, dann müssen wir weniger schreiben.

Der IQ-Mischer besteht aus zwei Multiplizierern.

Diese beiden Multiplizierer sind an einen lokalen Oszillator angeschlossen, der mit der Kreisfrequenz $ \omega_{0} $ schwingt und der 2 Ausgänge hat: Der eine Multiplizierer, nennen wir ihn den I-Multiplizierer, ist am 0°-Ausgang des lokalen Oszillators angeschlossen und der andere Multiplizierer, der Q-Multiplizierer, am 90°-Ausgang.

Die Amplituden sollen auch erst mal wieder keine Rolle spielen und werden der Einfachheit halber auf 1 gesetzt.

Das LO-Signal am I-Multiplizierer soll also 

$ U_{LO,I}(t) = \cos ( \omega_{0} t ) $

sein und das Signal am Q-Multiplizierer soll sein

$ U_{LO,Q}(t) = \cos (\omega_{0} t + 90° ) = - \sin ( \omega_{0} t ) $


Am Ausgang der Multiplizierer erscheinen dann die Produkte I und Q:

$ I = \cos ( \omega_{1} t ) \cos ( \omega_{0} t ) $

$ Q = - \cos ( \omega_{1} t ) \sin ( \omega_{0} t ) $

Jetzt wenden wir zwei Theoreme für Produkte von Winkelfunktionen an, nämlich 

$ \cos x \; \cos y = \frac{1}{2}\Big(\cos (x-y) + \cos (x+y)\Big) $  

und

$ \sin x \; \cos y = \frac{1}{2}\Big(\sin (x-y) + \sin (x+y)\Big) $

und erhalten

$ I = \frac{1}{2} \cos ( \omega_{1} + \omega_{0} )t + \frac{1}{2} \cos ( \omega_{1} - \omega_{0} )t $

$ Q = - \frac{1}{2} \sin ( \omega_{1} + \omega_{0} )t + \frac{1}{2} \sin ( \omega_{1} - \omega_{0} )t $


## So funktioniert SSB-Demodulation mit IQ-Signalen ##

Um aus dem IQ-Signal, das von einem SSB-modulierten Sender stammt, das ursprüngliche Modulationssignal zu demodulieren, muss man das Q-Signal um 90° in der Phase verschieben.

Ein breitbandige Phasenverschiebung (mit der Bandbreite des Modulationssignals) ist mit analogen Schaltungstechniken aufwendig, aber möglich, z.B. mit audio phasing networks nach [W2WAM](https://en.wikipedia.org/wiki/Robert_B._Dome). In diesem Fall ist es vorteilhafter, das I-Signal um -45° zu schieben und das Q-Signal um +45°; wichtig ist nur die Phasendifferenz von Q nach I; die muss 90° sein.
In der digitalen Signalverarbeitung bewirkt man die Phasenverschiebung mit einem sog. Hilbert-Tranformator, der als FIR-Filter mit bestimmten [Koeffizienten](https://de.wikipedia.org/wiki/Hilbert-Transformation#Berechnung_mit_FIR-Filter) realisiert werden kann.
Auch im digitalen Fall muss man aufpassen: Der Hilbert-Transformator für das Q-Signal hat eine bestimmte Verzögerungszeit; auch das I-Signal muss um diese gleiche Zeit verzögert werden, was man ebenfalls mit einem FIR-Filter erreichen kann.  

Rechnerisch haben wir es dagegen bequem: Wir müssen nur 90° zum Q-Signal addieren und erhalten das Signal

$ Q' = - \frac{1}{2} \sin ( \omega_{1} t + \omega_{0} t + 90°) + \frac{1}{2} \sin ( \omega_{1} t - \omega_{0} t +90°) $

$ Q' = - \frac{1}{2} \cos ( \omega_{1} + \omega_{0} )t + \frac{1}{2} \cos ( \omega_{1} - \omega_{0} )t $


### LSB ###

Um das untere Seitenband zu erhalten, müssen wir nur noch das $ I $ - und das $ Q' $-Signal addieren:

$ LSB = I + Q' = \cos ( \omega_{1} - \omega_{0} )t $

### USB ###

Für das obere Seitenband müssen wir das $ Q' $ - Signal vom $ I $-Signal abziehen:

$ USB = I - Q' = \cos ( \omega_{1} + \omega_{0} )t $

## SSB-Demodulation bei Phasen- und Amplitudenfehlern ##

Bisher war alles perfekt: Die beiden Mischer funktionierten als perfekte Multiplizierer und die beiden Ausgänge des Lokaloszillators waren exakt um 90° phasenverschoben zueinander und lieferten exakt gleiche Amplituden.

Weder Analogmultiplizierer noch analoge Quadraturoszillatoren funktionieren aber so perfekt.

Um einen Phasenfehler zu simulieren, sollen die beiden Ausgänge des Oszillators jetzt nicht mehr exakt eine
Phasendifferenz von $ 90° $ besitzen, sondern eine Phasendifferenz von $ 90° + \Delta \phi $.

Um einen Amplitudenfehler zu simulieren, soll das LO-Signal am Q-Mischer eine etwas andere Amplitude haben als 1.

Insgesamt soll also das Signal, das am Q-Mischer anliegt, um $ 90 ° $ und ein bisschen was dazu ($ \Delta \phi$) phasenverschoben sein und eine Amplitude $ a $ besitzen, wobei $ a \approx 1 $ sein soll:

$ U_{LO,Q}(t) = a \cos( \omega_{0} t + 90° + \Delta \phi ) = -a \sin( \omega_{0} t + \Delta \phi ) $

wobei hier  $ \cos( x + 90° ) = - \sin x $ zur Anwendung kam.

Mit dem folgenden Code kann man die Signale des lokalen Oszillators plotten:
